In [1]:
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
import numpy as np 
import pandas as pd 
import random
import datetime as dt
from sklearn import linear_model 
from sklearn.metrics import mean_absolute_error
import plotly
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf
import math

Using TensorFlow backend.


In [2]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_val), y_val),
                m.score(X_train, y_train), m.score(X_val, y_val)]
    print(f"rmse train {res[0]}, rmse val {res[1]}, r^2 train {res[2]}, r^2 val {res[3]}")

In [3]:
df=pd.read_csv('E:/Datasets/stock-market-dataset/stocks/PFE.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1972-06-01,0.000000,0.859375,0.846354,0.859375,0.000594,2332800
1,1972-06-02,0.859375,0.861979,0.846354,0.848958,0.000587,1531200
2,1972-06-05,0.848958,0.851562,0.841146,0.846354,0.000585,2452800
3,1972-06-06,0.846354,0.869792,0.843750,0.864583,0.000598,2227200
4,1972-06-07,0.864583,0.864583,0.851562,0.864583,0.000598,979200
...,...,...,...,...,...,...,...
12059,2020-03-26,29.889999,31.990000,29.879999,31.750000,31.750000,41053100
12060,2020-03-27,30.799999,31.740000,30.660000,30.900000,30.900000,36328600
12061,2020-03-30,31.049999,32.849998,31.049999,32.669998,32.669998,28409400
12062,2020-03-31,32.400002,33.000000,32.240002,32.639999,32.639999,27956500


making new features

In [4]:
df['H-L']=df['High']-df['Low']
df['O-C']=df['Close']-df['Open']

In [5]:
#https://www.investopedia.com/terms/m/movingaverage.asp
def MA(dataset,days):
    n=days
    dataset['{} DAYS MA'.format(n)]=0
    for i in range(len(dataset)):
        dataset['{} DAYS MA'.format(n)].iloc[i]=sum(dataset['Close'].iloc[i-n:i])/n

In [6]:
MA(df,7)
MA(df,14)
MA(df,21)

C:\Users\oukle\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,14 DAYS MA,21 DAYS MA
0,1972-06-01,0.000000,0.859375,0.846354,0.859375,0.000594,2332800,0.013021,0.859375,0.000000,0.000000,0.000000
1,1972-06-02,0.859375,0.861979,0.846354,0.848958,0.000587,1531200,0.015625,-0.010417,0.000000,0.000000,0.000000
2,1972-06-05,0.848958,0.851562,0.841146,0.846354,0.000585,2452800,0.010417,-0.002604,0.000000,0.000000,0.000000
3,1972-06-06,0.846354,0.869792,0.843750,0.864583,0.000598,2227200,0.026042,0.018229,0.000000,0.000000,0.000000
4,1972-06-07,0.864583,0.864583,0.851562,0.864583,0.000598,979200,0.013021,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12059,2020-03-26,29.889999,31.990000,29.879999,31.750000,31.750000,41053100,2.110001,1.860001,30.270000,31.452857,32.553333
12060,2020-03-27,30.799999,31.740000,30.660000,30.900000,30.900000,36328600,1.080000,0.100000,30.211429,31.219286,32.411905
12061,2020-03-30,31.049999,32.849998,31.049999,32.669998,32.669998,28409400,1.799999,1.619999,30.002857,31.015000,32.259524
12062,2020-03-31,32.400002,33.000000,32.240002,32.639999,32.639999,27956500,0.759998,0.239998,30.324286,30.877857,32.223809


In [8]:
#https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/standard-deviation
import math
def std_dev(dataset,days):
    n=days
    dataset['{} DAYS STD DEV'.format(days)]=0
    dataset['sq(C-MA)']=(dataset['Close']-dataset['{} DAYS MA'.format(n)])**2
    for i in range(len(dataset)):
        dataset['{} DAYS STD DEV'.format(n)].iloc[i]=math.sqrt(sum(dataset['sq(C-MA)'].iloc[i-n:i])/n)
    dataset.drop('sq(C-MA)', axis=1,inplace=True)

In [9]:
std_dev(df,7)

C:\Users\oukle\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,14 DAYS MA,21 DAYS MA,7 DAYS STD DEV
0,1972-06-01,0.000000,0.859375,0.846354,0.859375,0.000594,2332800,0.013021,0.859375,0.000000,0.000000,0.000000,0.000000
1,1972-06-02,0.859375,0.861979,0.846354,0.848958,0.000587,1531200,0.015625,-0.010417,0.000000,0.000000,0.000000,0.000000
2,1972-06-05,0.848958,0.851562,0.841146,0.846354,0.000585,2452800,0.010417,-0.002604,0.000000,0.000000,0.000000,0.000000
3,1972-06-06,0.846354,0.869792,0.843750,0.864583,0.000598,2227200,0.026042,0.018229,0.000000,0.000000,0.000000,0.000000
4,1972-06-07,0.864583,0.864583,0.851562,0.864583,0.000598,979200,0.013021,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12059,2020-03-26,29.889999,31.990000,29.879999,31.750000,31.750000,41053100,2.110001,1.860001,30.270000,31.452857,32.553333,1.528536
12060,2020-03-27,30.799999,31.740000,30.660000,30.900000,30.900000,36328600,1.080000,0.100000,30.211429,31.219286,32.411905,1.617717
12061,2020-03-30,31.049999,32.849998,31.049999,32.669998,32.669998,28409400,1.799999,1.619999,30.002857,31.015000,32.259524,1.637748
12062,2020-03-31,32.400002,33.000000,32.240002,32.639999,32.639999,27956500,0.759998,0.239998,30.324286,30.877857,32.223809,1.824683


In [11]:
#finding index
start=df.loc[df['Date']=='2009-04-06']
end=df.loc[df['Date']=='2019-04-05']
split=df.loc[df['Date']=='2017-04-03']
print(start.index,end.index,split.index)

Int64Index([9297], dtype='int64') Int64Index([11814], dtype='int64') Int64Index([11309], dtype='int64')


In [12]:
X=df.iloc[:,-6:]
y=df['Close']

# Like paper

In [13]:
start_time, split_time, end_time=9297, 11309,11814
X_train,X_val=X.loc[start_time:split_time],X.loc[split_time:end_time]
y_train,y_val=y.loc[start_time:split_time],y.loc[split_time:end_time]

In [14]:
X_train

,H-L,O-C,7 DAYS MA,14 DAYS MA,21 DAYS MA,7 DAYS STD DEV
9297,0.650000,0.150000,13.864286,13.935000,13.763810,0.289123
9298,0.250000,-0.110000,13.768571,13.895714,13.810476,0.258804
9299,0.350000,-0.060000,13.692857,13.842857,13.852381,0.276566
9300,0.370000,-0.070000,13.661429,13.827143,13.870952,0.265912
9301,0.300000,-0.050000,13.651429,13.821429,13.907143,0.229396
...,...,...,...,...,...,...
11305,0.289997,0.099998,34.270000,34.239286,34.263810,0.180974
11306,0.340000,0.170002,34.247143,34.257143,34.258095,0.180849
11307,0.289997,0.160000,34.240001,34.277143,34.268095,0.178515
11308,0.220001,0.000000,34.254286,34.294286,34.264762,0.181674


In [15]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [16]:
y_val

11309    34.240002
11310    34.340000
11311    34.189999
11312    34.110001
11313    34.099998
           ...    
11810    42.840000
11811    42.910000
11812    42.840000
11813    42.740002
11814    42.990002
Name: Close, Length: 506, dtype: float64

In [17]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [18]:
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [19]:
y_pred=model.predict(X_val)

In [20]:
y_pred

array([34.27610104, 34.26180042, 34.20079998, 34.31450035, 34.32040024,
       34.25550007, 34.21930031, 34.27460037, 34.24340019, 34.21270016,
       34.2220002 , 33.81689972, 34.08099998, 34.13349995, 33.47970047,
       33.68710045, 33.94560055, 33.57660095, 34.02950047, 33.56390076,
       33.98520016, 34.01230034, 34.02200024, 34.01520016, 33.98150021,
       33.26390003, 33.66509983, 33.48249969, 33.3493993 , 33.35719967,
       33.41020016, 32.69139978, 32.88540007, 32.74339985, 32.33850027,
       32.52980011, 32.34910036, 31.99669975, 31.92159983, 31.91899969,
       32.06469954, 32.04499979, 32.04939972, 32.3691996 , 32.35409952,
       32.37839975, 32.27720015, 33.19499968, 32.37699942, 32.33399975,
       32.52290012, 32.78790031, 32.30900002, 32.72360004, 32.8269998 ,
       33.08759974, 33.44130005, 33.68319954, 33.75629963, 33.69609989,
       33.79290009, 33.89389992, 33.97549999, 34.37619995, 33.42440014,
       33.40050045, 33.94510002, 33.82210003, 33.16149956, 33.26

In [21]:
y_val

11309    34.240002
11310    34.340000
11311    34.189999
11312    34.110001
11313    34.099998
           ...    
11810    42.840000
11811    42.910000
11812    42.840000
11813    42.740002
11814    42.990002
Name: Close, Length: 506, dtype: float64

In [22]:
rmse(y_pred,y_val)

4.187778104327143

In [23]:
print_score(model)

rmse train 0.15767258553684282, rmse val 4.187778104327143, r^2 train 0.9994638469106586, r^2 val -0.17970459033886277


# suffled

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2)

In [25]:
y_train

5408      5.197917
8245     24.160000
6491     26.916666
8918     25.080000
10523    31.480000
           ...    
9676     17.260000
329       0.940104
434       0.809896
119       0.895833
10193    25.190001
Name: Close, Length: 9651, dtype: float64

In [26]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [27]:
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [28]:
y_pred=model.predict(X_val)

In [29]:
rmse(y_pred,y_val)

0.5353054398674824

In [30]:
model.score(X_val,y_val)

0.998717394996211

In [31]:
print_score(model)

rmse train 0.19104338317964614, rmse val 0.5353054398674824, r^2 train 0.9998336736611092, r^2 val 0.998717394996211
